## Collecting Yelp Reviews

In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
import time

In [14]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
start_page = 420
#url="https://www.yelp.ca/biz/meet-on-main-vancouver?start="+str(start_page)+"&sort_by=date_desc"
url = "https://www.yelp.ca/biz/meet-on-main-vancouver?"
response = requests.get(url, headers=headers)

In [15]:
content = response.content
soup = BeautifulSoup(content,"lxml")

In [16]:
reviews = json.loads(soup.find('script', type='application/ld+json').string)['review']
reviews[0]

{'author': 'Tyson L.',
 'datePublished': '2021-06-26',
 'reviewRating': {'ratingValue': 5},
 'description': 'Rarely will you find MeeT empty -- an arbitrary indicator of popularity if you care about that -- as the vegan food is of high quality with solid portions. \n\nAfter multiple visits, some with Vegans, others with the &quot;oh but you need animal protein to survive cohort&quot;, 90% of guests would recommend the restaurant for its all-inclusive, varying menu.\n\nMy test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes, and I could not with the same flavors.\n\nPersonally a big fan of the Macro bowl or the Meet burger, but most friends are content with the selection of poutines and the Mighty Mac.\n\nGood quality food is good quality food, and whether you&apos;re vegan or not you will be satisfied'}

In [55]:
sample_rev = reviews[0]['description']
sample_rev

'Rarely will you find MeeT empty -- an arbitrary indicator of popularity if you care about that -- as the vegan food is of high quality with solid portions. \n\nAfter multiple visits, some with Vegans, others with the &quot;oh but you need animal protein to survive cohort&quot;, 90% of guests would recommend the restaurant for its all-inclusive, varying menu.\n\nMy test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes, and I could not with the same flavors.\n\nPersonally a big fan of the Macro bowl or the Meet burger, but most friends are content with the selection of poutines and the Mighty Mac.\n\nGood quality food is good quality food, and whether you&apos;re vegan or not you will be satisfied'

In [2]:
regex = r'&.{4};'
unwanted = set()
punc = '''!()-[]{};:'"\,<>./?@#$^&*_~'''

In [3]:
for i in re.finditer(regex, sample_rev):
    unwanted.add(i.group())

NameError: name 'sample_rev' is not defined

In [4]:
def clean_sent(text):
    cleaned = []
    for word in text.split():
        if not word.isalpha():
            for unwanted_string in unwanted:
                if unwanted_string in word:
                    word = word.replace(unwanted_string, "")
            word = word.strip(punc)
            cleaned.append(word)
        else:
            cleaned.append(word)
    
    return " ".join(cleaned)

In [58]:
clean_sent(sample_rev)

'Rarely will you find MeeT empty  an arbitrary indicator of popularity if you care about that  as the vegan food is of high quality with solid portions After multiple visits some with Vegans others with the oh but you need animal protein to survive cohort 90% of guests would recommend the restaurant for its all-inclusive varying menu My test for a good restaurant is whether I could replicate the dishes and/or be willing to create the dishes and I could not with the same flavors Personally a big fan of the Macro bowl or the Meet burger but most friends are content with the selection of poutines and the Mighty Mac Good quality food is good quality food and whether youre vegan or not you will be satisfied'

In [32]:
def get_rating(rating_dict):
    return rating_dict['ratingValue']

In [60]:
yelp_df = pd.DataFrame.from_dict(reviews)
yelp_df['reviewRating'] = yelp_df['reviewRating'].apply(get_rating)
yelp_df['description'] = yelp_df['description'].apply(clean_sent)
yelp_df

,author,datePublished,reviewRating,description
0,Tyson L.,2021-06-26,5,Rarely will you find MeeT empty an arbitrary ...
1,Melissa E.,2021-10-09,4,Im not a vegan but I love trying innovative ve...
2,Matt B.,2021-07-30,4,Im much more sold on this place nowadays I ord...
3,Jill N.,2021-03-18,5,Went here for St Patties Day Our server Derek ...
4,Maling S.,2020-09-17,5,Sept 17 and went with my brother for vegan foo...
5,Dana S.,2021-02-18,5,Ive ordered takeout from the Yaletown location...
6,Michelle H.,2020-05-08,4,I havent had this is YEARS I used to be SO obs...
7,Jasmine Y.,2021-09-15,5,Ordered takeout online to the Main Street loca...
8,Aaron W.,2021-05-24,1,Saw 3 patio tables empty and barely any line u...
9,Jennifer B.,2020-03-12,5,Love the MeeT restaurants in Vancouver Ive eat...


In [141]:
yelp_df.to_csv('yelp_revs.csv', index=False)

### Collecting reviews from multiple restaurants

Get the links from different restaurants. Here, I searched for Vegan restaurants in Vancouver.

In [25]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
start_page = 420
#url = "https://www.yelp.ca/search?find_desc=Vegan%20Restaurants&find_loc=Vancouver%2C%20BC"
urls = ["https://www.yelp.ca/search?find_desc=Vegan%20Restaurants&find_loc=Vancouver%2C%20BC", "https://www.yelp.ca/search?find_desc=vegan%20Restaurants&find_loc=Vancouver%2C%20BC&start=10", "https://www.yelp.ca/search?find_desc=vegan%20Restaurants&find_loc=Vancouver%2C%20BC&start=20"]
#response = requests.get(url, headers=headers)

In [18]:
response = requests.get(url, headers=headers)
content = response.content
soup = BeautifulSoup(content,"lxml")

In [26]:
resto_links = []

for url in urls:
    response = requests.get(url, headers=headers)
    content = response.content
    soup = BeautifulSoup(content,"lxml")
    for a in soup.find_all('a', {"class": "css-1422juy"}): ## css changes regularly, inspect element again!
        if a['href'][1:4] == 'biz':
            resto_links.append((a['name'], a['href']))
        
resto_links

[('MILA Plant-Based', '/biz/mila-plant-based-vancouver?osq=Vegan+Restaurants'),
 ('Lotus Seed Vegan', '/biz/lotus-seed-vegan-vancouver?osq=Vegan+Restaurants'),
 ('MeeT in Gastown', '/biz/meet-in-gastown-vancouver?osq=Vegan+Restaurants'),
 ('Vegan Cave Cafe', '/biz/vegan-cave-cafe-vancouver?osq=Vegan+Restaurants'),
 ('BeetBox', '/biz/beetbox-vancouver?osq=Vegan+Restaurants'),
 ('Nuba in Gastown', '/biz/nuba-in-gastown-vancouver-2?osq=Vegan+Restaurants'),
 ('Do Chay', '/biz/do-chay-vancouver-2?osq=Vegan+Restaurants'),
 ('MeeT on Main', '/biz/meet-on-main-vancouver?osq=Vegan+Restaurants'),
 ('CHAU Veggie Express',
  '/biz/chau-veggie-express-vancouver-2?osq=Vegan+Restaurants'),
 ('Tama Organic Life',
  '/biz/tama-organic-life-vancouver?osq=Vegan+Restaurants'),
 ('Arike Restaurant',
  '/biz/arike-restaurant-vancouver-2?osq=vegan+Restaurants'),
 ('4 Stones Vegetarian Cuisine',
  '/biz/4-stones-vegetarian-cuisine-richmond?osq=vegan+Restaurants'),
 ('Buddha-Full', '/biz/buddha-full-north-vanc

Now we do the same thing as Part 1 but we will get the reviews from each of the links instead of just one link.

In [27]:
resto_revs = []
for name, link in resto_links:
    url = 'https://www.yelp.ca' + link
    response = requests.get(url, headers=headers)
    content = response.content
    soup = BeautifulSoup(content, "lxml")
    reviews = json.loads(soup.find('script', type='application/ld+json').string)['review']
    for rev in reviews:
        rev['name'] = name
        
    resto_revs.extend(reviews)

In [28]:
all_reviews = ""
for rev in resto_revs:
    all_reviews = all_reviews + rev['description'] + " "

In [29]:
regex = r'(&.{4};|&.{4})'
unwanted = set()
punc = '''!()-[]{};:'"\,<>./?@#$^&*_~'''

In [30]:
for i in re.finditer(regex, all_reviews):
    unwanted.add(i.group())
    
unwanted

{'&amp;', '&apos;', '&quot;'}

In [33]:
yelp_df = pd.DataFrame.from_dict(resto_revs)
yelp_df['reviewRating'] = yelp_df['reviewRating'].apply(get_rating)
yelp_df['description'] = yelp_df['description'].apply(clean_sent)
yelp_df

,author,datePublished,reviewRating,description,name
0,Jess S.,2021-07-05,5,Everything from the atmosphere to the service ...,MILA Plant-Based
1,Arlene B.,2021-10-03,4,You dont have to be vegan to appreciate the fo...,MILA Plant-Based
2,Poonam M.,2021-08-08,4,This the old Juniper so the layout is the same...,MILA Plant-Based
3,Suzan L.,2021-06-26,1,Very disappointing experience The Sakura cockt...,MILA Plant-Based
4,Sangeetha M.,2021-05-05,4,I ordered for delivery so I cant comment on th...,MILA Plant-Based
...,...,...,...,...,...
581,Mark C.,2016-03-15,5,Great people who really care about Vegans choi...,Vegan Pizza House
582,April T.,2019-05-27,5,Great pizza this is the only place where I lik...,Vegan Pizza House
583,Patricia L.,2017-08-09,4,My go-to for vegan take out in Vancouver They ...,Vegan Pizza House
584,Steven S.,2017-12-29,4,Ive become vegan recently so we wandered into ...,Vegan Pizza House


In [34]:
yelp_df.to_csv('yelp_revs.csv', index=False)

## EDA

In [35]:
yelp_df['reviewRating'].value_counts()

5    315
4    156
2     41
3     40
1     34
Name: reviewRating, dtype: int64

In [36]:
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords
from collections import defaultdict

In [37]:
lemmatizer = WordNetLemmatizer()
vocab = defaultdict(int)
for review in yelp_df['description']:
    for word in word_tokenize(review):
        if word.lower() not in stopwords.words('english'):
            lemma = lemmatizer.lemmatize(word.lower())
            vocab[lemma] += 1

In [38]:
sorted(vocab, key=vocab.get, reverse=True)[:10]

['food',
 'good',
 'place',
 'vegan',
 'like',
 'really',
 'also',
 'delicious',
 'great',
 'time']

In [39]:
sorted(vocab, key=vocab.get)[:10]

['calen',
 'toddler',
 'navigate',
 'appropriate',
 'settled',
 'agedashi',
 '+3',
 'puffed',
 'maldon',
 'indigenous']